In [0]:
## Created a new notebook and loaded the e - commerce data set directly from kaggle to databricks workspace

In [0]:
#installing kaggle (This command installs the Kaggle Python library so that you can download datasets from Kaggle directly inside your Databricks notebook.)

!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 102.5 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
#Configure kaggle credentials -(Kaggle access is ready)

import os

os.environ["KAGGLE_USERNAME"] = "vennusanjana"
os.environ["KAGGLE_KEY"] = "a0847d4801e29f9710cf42523413bb8b"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
#Creating data base schema

spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

In [0]:
#create volume for data storage 

spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

In [0]:
### Downloaded data set from kaggle 

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
ecommerce-behavior-data-from-multi-category-store.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
### extract downloaded dataset 

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-f0ee36f5-fe11-4af4-aae7-6d nogroup 8.4G Jan 12 08:19 2019-Nov.csv
-rwxrwxrwx 1 spark-f0ee36f5-fe11-4af4-aae7-6d nogroup 5.3G Jan 12 08:21 2019-Oct.csv
-rwxrwxrwx 1 nobody                           nogroup 4.3G Jan 12 07:48 ecommerce-behavior-data-from-multi-category-store.zip


In [0]:
### clean up zip file 

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 spark-f0ee36f5-fe11-4af4-aae7-6d nogroup 8.4G Jan 12 08:19 2019-Nov.csv
-rwxrwxrwx 1 spark-f0ee36f5-fe11-4af4-aae7-6d nogroup 5.3G Jan 12 08:21 2019-Oct.csv


In [0]:
### restart python environment 


In [0]:
%restart_python

In [0]:
#Load November 2019 data 

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

In [0]:
## Load October 2019 data 

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

In [0]:
### Display dataset statistics and schema

In [0]:
### verifying is file exists 

dbutils.fs.ls("/Volumes/workspace/ecommerce/ecommerce_data")

[FileInfo(path='dbfs:/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv', name='2019-Nov.csv', size=9006762395, modificationTime=1768205969000),
 FileInfo(path='dbfs:/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv', name='2019-Oct.csv', size=5668612855, modificationTime=1768206113000)]

In [0]:
df = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    inferSchema=True
)

In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()

October 2019 - Total Events: 42,448,764

SCHEMA:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
### display sample data 

In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
df.show(5, truncate=False)


SAMPLE DATA (First 5 rows):
+-------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 00:00:00|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.2   |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00:01|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |NULL    |543.1  |519107250|566511c2-e2e3-422b-b695-c

In [0]:
### loading nov data 

df_nov = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    header=True,
    inferSchema=True
)


In [0]:
### display 10 records in nov month 

df_nov.limit(10).display()

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-01T00:00:00.000Z,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2019-11-01T00:00:00.000Z,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2019-11-01T00:00:01.000Z,view,17302664,2053013553853497655,null,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
2019-11-01T00:00:01.000Z,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
2019-11-01T00:00:01.000Z,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
2019-11-01T00:00:01.000Z,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
2019-11-01T00:00:01.000Z,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
2019-11-01T00:00:02.000Z,view,15900065,2053013558190408249,null,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
2019-11-01T00:00:02.000Z,view,12708937,2053013553559896355,null,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24
2019-11-01T00:00:02.000Z,view,1004258,2053013555631882655,electronics.smartphone,apple,732.07,532647354,d2d3d2c6-631d-489e-9fb5-06f340b85be0


In [0]:
# resuable code to run the e-commerce data set (reusable notebook)

BASE_PATH = "/Volumes/workspace/ecommerce/ecommerce_data"

df = spark.read.option("header", True).option("inferSchema", True).csv(
    f"{BASE_PATH}/*.csv"
)